# ROBOMYSTIC: Automated Investing that Feels Human

### 1) Client Onboarding and Know Your Client Criteria
##### Our company compiles customer data by having prospective clients fill out an intake form. Although our mock-up doesn't fulfill all the 'Know Your Client' (KYC) criteria as required by Canadian law, we have approximated to simulate how KYC data would be stored and imported into our automated system to pair clients with the appropriate portfolio. We have chosen the .YAML file format for its versatility and the ability it gives us to iterate through an (hopefully for our firm) ever-increasing number of client forms automatically. Future versions of our software would include identity verification technology, a more robust suitability criterion by which we might reject prospective clients based on, for example, creditworthiness or anti-money laundering grounds.

In [2]:
# Import Libraries and Modules
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
%matplotlib inline
import requests
import urllib.request 
import os
import json
from pandas.io.json import json_normalize
import seaborn as sns
import yfinance as yf
from yahoofinancials import YahooFinancials
from datetime import date
from pathlib import Path
from dotenv import load_dotenv
from datetime import date
from pandas_datareader import data as pdr
import numpy as np
import seaborn as sns
import yaml
import io
import pytz
import glob
import holoviews as hv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation

yf.pdr_override()

ModuleNotFoundError: No module named 'alpaca_trade_api'

In [ ]:
# Load .env enviroment variables
load_dotenv()

In [ ]:
# Set API keys and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
quandl_api_key = os.getenv("QUANDL_API_KEY")
rapid_api_key = os.getenv("RAPID_API_KEY")

In [ ]:
## Set file path and import all .yaml files into Pandas DataFrame
path = r"C:\Users\berny\Desktop\Fintechbootcamp\HUMAN ROBOT\BC_Project_1\BC_Project_1"
all_files = glob.glob(path + "/*.yaml")
li = []
for filename in all_files[:10]:
    with open(filename, "r") as fh:
        df = pd.json_normalize(yaml.safe_load(fh.read()))
    li.append(df)
all_cust_kyc = pd.concat(li)
columns = ["Date", "First Name", "Last Name", "Birth Date", "Annual Income", "Initial Investment", "Occupation", "Investment Goals", "Investment Time Horizon", "Risk Tolerance", "Investment Constraints"]
all_cust_kyc.columns = columns
all_cust_kyc = all_cust_kyc.set_index(all_cust_kyc["Last Name"])

all_cust_kyc = all_cust_kyc.sort_index()
all_cust_kyc

In [ ]:
# Access Columns in Dataframe
risk = all_cust_kyc["Risk Tolerance"]
income = all_cust_kyc["Annual Income"]
goals = all_cust_kyc["Investment Goals"]
horizon = all_cust_kyc["Investment Time Horizon"]
initial_investment = all_cust_kyc["Initial Investment"]

In [ ]:
# Set customer last name
customer = "Sher"

In [ ]:
# Iterate through data given content of 'customer'
cust = all_cust_kyc["Last Name"]

for name in cust:
    if customer == cust[0]:
        r = risk[0] 
        i = income[0]
        g = goals[0]
        h = horizon[0]
    elif customer == cust[1]:
        r = risk[1] 
        i = income[1]
        g = goals[1]
        h = horizon[1]
    elif customer == cust[2]:
        r = risk[2] 
        i = income[2]
        g = goals[2]
        h = horizon[2]
    elif customer == cust[3]:
        r = risk[3] 
        i = income[3]
        g = goals[3]
        h = horizon[3]
    elif customer == cust[4]:
        r = risk[4] 
        i = income[4]
        g = goals[4]
        h = horizon[4]
    
print(f"{customer}: r = {r}, i = {i}, g = {g}, h = {h}")

In [ ]:
## Categorize client's by assigning numerical value to Risk Tolerance (r)/ Annual Income (i) / Investment Goals (g)/ Time Horizon (h) 
if r == "low":
    assign_r = 0
elif r == "medium":
    assign_r = 1
elif r == "high":
    assign_r = 2

if i <= 75000:
    assign_i = 0
elif i >= 75001 and income <= 150000:
    assign_i = 1
elif i >= 150001:
    assign_i = 2

## Categorize client's Investment Goals
if g == "preservation":
    assign_g = 0
elif g == "income":
    assign_g = 1
elif g == "growth":
    assign_g = 2
    
if h >= 0 and h <= 10:
    assign_h = 3
elif h >= 11 and h <= 20:
    assign_h = 2
elif h >= 21:
    assign_h = 1
    
print(f"{customer}: r = {r}, i = {i}, g = {g}, h = {h}")

In [ ]:
## Compile client categorizations to determine portfolio suitability
pn = assign_r + assign_i + assign_g + assign_h
if pn >= 0 and pn <= 2.5:
    assign_portfolio = 1
if pn >= 3 and pn <= 5:
    assign_portfolio = 2
if pn >= 5.5: 
    assign_portfolio = 3
print(f"{customer} Portfolio Number is {pn}, assign {customer} to Portfolio {assign_portfolio}")

In [ ]:
# Determine portfolio assignment outcome for subsequent analysis 
if pn == 1:
    portfolio = portfolio_1
elif pn == 2:
    portfolio = portfolio_2
elif pn == 3:
    portfolio = portfolio_3

### 2) Import and Clean Stock Data for Fundamental and Statistical Analysis
##### Our company has selected top performing stocks from the S&P 500 Index as our core portfolio. For these companies, we import important fundamental data garnered from the companies' balance sheets. Although we have not yet fully integrated this information into our analysis, we include this data now because proper due diligence will eventually require performing fundamental ratio analysis that will give us a better portrait of the overall performance of these companies, and also of their performance compared to market, sector, and industry peers and competitors. For now we have limited our fundamental analysis to Price-Earnings (P/E) Ratios. Our stock selections comes as a result of many years of analysis, market research, and market participation. Our core portfolio selection is composed of stocks in which we have been invested for more than 20 years, hence our choice of 20 years of historical performance data. Along the way, we have added 'SHOP', 'PYPL', and 'DAL', as they have fit our rigorous criteria for adoption. 

In [ ]:
# Set timeframe (YYYY-MM-DD)
start_date = '2000-01-01'
today = date.today()

In [ ]:
# Set Ticker list
ticker_list = ['SPY','AAPL','AMZN','DAL','GE','JNJ','MSFT','MNST','PYPL','SHOP','UPS']

In [ ]:
#function that grabs a Yahoo Finance JSON URL and outputs the results as a dictionary
#Code Ref: https://marqueegroup.ca/resource/how-to-use-python-in-a-finance-environment/

def fnYFinJSON(stock):
  urlData = "https://query2.finance.yahoo.com/v7/finance/quote?symbols="+stock
  webUrl = urllib.request.urlopen(urlData)
  if (webUrl.getcode() == 200):
    data = webUrl.read()
  else:
      print ("Received an error from server, cannot retrieve results " + str(webUrl.getcode()))
  yFinJSON = json.loads(data)
  return yFinJSON["quoteResponse"]["result"][0]

In [ ]:
# Create a table based on tickers and company info fields needed
tickers = ticker_list
fields = {'shortName':'Company Name',  
          'fullExchangeName':'Exchange', 
          'marketCap':'Market Cap',
          'regularMarketPrice':'Price',
          'fiftyTwoWeekLow':'52-Wk Low', 
          'fiftyTwoWeekHigh':'52-Wk High',
          'regularMarketDayHigh':'High', 
          'regularMarketDayLow':'Low',
          'regularMarketVolume': 'Volume',
          'averageDailyVolume3Month': 'Ave. Quarterly Vol.',
          'epsTrailingTwelveMonths':'EPS(TTM)',
          'trailingPE': 'PE Ratio (TTM)'
         }
results = {}
for ticker in tickers:
  tickerData = fnYFinJSON(ticker)
  singleResult = {}
  for key in fields.keys():
    if key in tickerData:
      singleResult[fields[key]] = tickerData[key]
    else:
      singleResult[fields[key]] = "N/A"
  results[ticker] = singleResult

In [ ]:
# Arrange index according to a specific order
# Code Ref:https://stackoverflow.com/questions/55397178/dataframe-creation-from-dict-index-order

co_info_label = ['Company Name', 
                 'Exchange', 
                 'Market Cap',
                 'Price',
                 'Low',
                 'High', 
                 '52-Wk Low', 
                 '52-Wk High',
                 'Volume',
                 'Ave. Quarterly Vol.',
                 'EPS(TTM)',
                 'PE Ratio (TTM)',
                ]

In [ ]:
company_info_df = pd.DataFrame.from_dict(results)
company_info_df = company_info_df.reindex(co_info_label)
company_info_df

In [ ]:
# YahooFinancials - Revenue 
yahoo_financials_tickers = YahooFinancials(ticker_list)
earnings = yahoo_financials_tickers.get_stock_earnings_data(reformat=True)

### 3) Import and Cleaning Data for PE Ratios:
##### To acquire results of companies history of PE Ratios to determine the strenth of the stock.

In [ ]:
#import csv data for 
AAPL_PE_path  = Path("AAPL Comparison-to-Industry.csv")
all_historical_PE_path = Path("all_stocks_historical_PE_ratio.csv")  


In [ ]:
#read dataframe
AAPL_PE_df  = pd.read_csv(AAPL_PE_path, parse_dates=True, infer_datetime_format=True)
AAPL_PE_df.head()

In [ ]:
#display colomns name
AAPL_PE_df.columns

In [ ]:
#change colomns name
columns = ["Date", "Apple Inc.", "Technology"]

In [ ]:
#dispaly new colomns name
AAPL_PE_df.columns = columns
AAPL_PE_df.head()

In [ ]:
#display table from row 3 to the end
AAPL_PE_df = AAPL_PE_df[3:]

In [ ]:
#read dataframe
AAPL_PE_df.head()

In [ ]:
#change datatype frome object to datetime64[ns]
AAPL_PE_df['Date'] = AAPL_PE_df['Date'].astype('datetime64[ns]')
AAPL_PE_df.dtypes

In [ ]:
#change datatype from object to float64
AAPL_PE_df['Apple Inc.'] = AAPL_PE_df['Apple Inc.'].astype('float')
AAPL_PE_df['Technology'] = AAPL_PE_df['Technology'].astype('float')
AAPL_PE_df.dtypes

In [ ]:
#set "Date" as index
AAPL_PE_df = AAPL_PE_df.set_index(AAPL_PE_df["Date"])
AAPL_PE_df.head()

In [ ]:
#delete "date" columne
AAPL_PE_df = AAPL_PE_df.drop(columns=["Date"])

In [ ]:
#sort date
AAPL_PE_df.sort_index(inplace=True)

In [ ]:
#display data frame
AAPL_PE_df

In [ ]:
#plot data frame
AAPL_PE=AAPL_PE_df.hvplot(
                width=750, height=450,rot=45,
                title="Apple Inc., P/E, long-term trends, comparison to industry (technology)")
AAPL_PE

In [ ]:
#read dataframe
all_historical_PE_df  = pd.read_csv(all_historical_PE_path, parse_dates=True, infer_datetime_format=True)
all_historical_PE_df.head()

In [ ]:
#delete columne "Date"
all_historical_PE_df = all_historical_PE_df.drop(columns=["Date"])

In [ ]:
#sort index
all_historical_PE_df.sort_index(inplace=True)

In [ ]:
all_historical_PE_df.columns

In [ ]:
#rename columns
columns = ["AAPL","AMZN","DAL","GE","JNJ","MSFT","MNST","PYPL","SHOP","UPS","SPY"]

In [ ]:
#set new columns name
all_historical_PE_df.columns = columns
all_historical_PE_df.head()

In [ ]:
#change 'SPY' data from object to float
all_historical_PE_df['SPY'] = all_historical_PE_df['SPY'].astype('float')
all_historical_PE_df.dtypes

In [ ]:
#plot dataframe
historical_PE = all_historical_PE_df.hvplot(
                y = ['AAPL', "AMZN",'DAL', 'GE', 'JNJ',"MSFT","MNST","PYPL","SHOP","UPS","SPY"],
                width=950, height=450,rot=90,
                title="P/E, long-term trends, comparison to S&P 500")  
historical_PE


### 4) Conduct Statistical Analysis on Historical Stock Data to Allocate Stocks to Portfolios
##### We conduct statistical analysis on our stock data with the intention of using these numbers to organize our stocks into their respective portfolio designations to suit the categories into which we have organized our portfolio offerings. The three criteria we have chosen for allocating stocks to these respective portfolios are annualized  Standard Deviation, Cumulative Retunrs, Cumulative Risk, Beta ,and Sharpe Ratio.

In [ ]:
# Download historical statistical data = 20 years
## Import Stock Data
stat_data_20yr = yf.download("SPY AAPL AMZN DAL GE JNJ MSFT MNST PYPL SHOP UPS", start="2000-01-01", end=today).Close
stat_data_20yr.head()

In [ ]:
# Code Ref: https://stackoverflow.com/questions/38133064/get-adj-close-using-pandas-datareader
stocks_adjclose_20yr = yf.download(ticker_list,  start='2000-01-01', end = today, period = "1d")['Adj Close']
stocks_adjclose_20yr = stocks_adjclose_20yr.fillna(0)
stocks_adjclose_20yr.head()

In [ ]:
# Calculate daily returns
daily_returns_20yr = stocks_adjclose_20yr.pct_change()
daily_returns_20yr = daily_returns_20yr.fillna(0)
daily_returns_20yr.head()

In [ ]:
## Calculate the standard deviation
daily_returns_20yr_std = daily_returns_20yr.std()
daily_returns_20yr_std = daily_returns_20yr_std.sort_values()
daily_returns_20yr_std

In [ ]:
# Average Daily Returns
daily_returns_20yr_mean = daily_returns_20yr.mean().round(5)
print(daily_returns_20yr_mean)
daily_returns_20yr_mean.head().round(5)

In [ ]:
## Calculate the annualized standard deviation
ann_portfolio_std = daily_returns_20yr_std * np.sqrt(252)
ann_portfolio_std = ann_portfolio_std.sort_values()
ann_portfolio_std

In [ ]:
# Calculate cumulative returns of all portfolios
cumulative_returns = (1+daily_returns_20yr).cumprod() -1
cumulative_returns.head()

In [ ]:
cumulative_std = cumulative_returns.std()
print(cumulative_std)

In [ ]:
# Determine which portfolios are riskier than the S&P 500
cumulative_risk = cumulative_std[cumulative_std > cumulative_std["SPY"]]
print(cumulative_risk)

In [ ]:
# Calculate the annualized standard deviation (252 trading days)
annual_std = cumulative_std * np.sqrt(252)
annual_std

In [ ]:
sharpe_ratios = (cumulative_returns.mean() * 252) / (cumulative_returns.std() * np.sqrt(252))
sharpe_ratios.sort_values()

##### We have now compiled our stock information sufficiently to be able to allocate these stocks to our respective portfolios. Below we define our specific portfolios and begin analysis on them so that we will be able to provide our client with sufficient information and visualizations that they can be confident they have had an appropriate portfolio selected for them. 

In [ ]:
### We need code to extract the P/E Ratios, sharpe ratios, and annualized standard deviation for each data set. 
# if sharpe_ratio <= . . . etc.

In [ ]:
portfolio_1 = stocks.drop(columns=["SPY", "SHOP", "DAL", "GE", "PYPL"]) 
portfolio_2 = stocks.drop(columns=["SPY", "JNJ", "UPS", "SHOP", "DAL"])
portfolio_3 = stocks.drop(columns=["SPY", "JNJ", "UPS", "MSFT", "MNST"])

In [ ]:

# Portfolio return, volatility, Sharpe Ratio

annualized_return = ((np.mean(portfolio['Total'].pct_change() + 1)) ** 252 - 1)

annualized_vol = np.std(portfolio['Total'].pct_change()) * np.sqrt(252)

Sharpe_Ratio = (annualized_return - 0.025) / annualized_vol  # 2.5% risk-free rate

print('Our portfolio delivered a', "{0:.1%}".format(annualized_return), 
      'annual return, with a', "{0:.1%}".format(annualized_vol), 
      'volatility rate, resulting in a Sharpe Ratio of', "{0:.2f}".format(Sharpe_Ratio))

# 5) Perform Monte Carlo Simulation to Predict the Profitability for Client's Portfolio According to Investment Time Horizon
##### We are in the process of determining another way to get a baseline dataframe arranged that allows us to conduct a Monte Carlo analysis without being limited by the alpaca API's 1000 object limit per call.

In [ ]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    base_url="https://paper-api.alpaca.markets",
    api_version="v2")

In [ ]:
# Format current date as ISO format

today = pd.Timestamp("2021-04-09", tz="America/New_York").isoformat() 


# Set the tickers
tickers = ["SPY","AGG"]

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get current closing prices for SPY and AGG

df_portfolio = alpaca.get_barset(
    tickers,
    timeframe,
    start = today,
    end = today
).df


# Preview DataFrame
df_portfolio

In [ ]:
# Set start and end dates of five years back from today.
# Sample results may vary from the solution based on the time frame chosen
start_date = pd.Timestamp('2000-01-01', tz='America/New_York').isoformat()
end_date = pd.Timestamp('today', tz='America/New_York').isoformat()

In [ ]:
# Get 5 years' worth of historical data for SPY and AGG
df_stock_data = alpaca.get_barset(
    tickers,
    timeframe,
    start=start_date,
    end=end_date,limit=1000
).df


# Display sample data
df_stock_data.head()

In [ ]:
# Configuring a Monte Carlo simulation to forecast 20 years cumulative returns
MC_20_yrs = MCSimulation(
    portfolio_data = df_stock_data,
    num_simulation = 500,
    num_trading_days = 252*20
)

In [ ]:
# Printing the simulation input data
MC_20_yrs.portfolio_data.head()

In [ ]:
# Running a Monte Carlo simulation to forecast 20 years cumulative returns
MC_20_yrs.calc_cumulative_return()

In [ ]:
# Plot simulation outcomes
line_plot = MC_20_yrs.plot_simulation()

# Save the plot for future usage
line_plot.get_figure().savefig("MC_20_yr_sim_plot.png", bbox_inches="tight")


In [ ]:
# Plot probability distribution and confidence intervals
dist_plot = MC_20_yrs.plot_distribution()
#dist_chart = dist_plot.hvplot(title='Distribution of Final Cumulative Returns Across All 500 Simulations', height = 450, width = 750)
#dist_chart

# 6) Visualizations, Panel, and Customer Dashboard
##### Our HumanRobo now compile representations of our analysis to provide the client with an interactive dashboard by which they can see the analytics of their respective portfolio.

In [ ]:
stat_data_20yr.hvplot.line(title='All Stocks Closing Prices in 20 years',height = 450, width = 900)

In [ ]:
#daily_returns_20yr.plot(figsize = (20, 10))
#plt.title('Daily Returns')
#plt.ylabel('Percent')
daily_returns_20yr_plot = daily_returns_20yr.hvplot.line(title='All Stocks 20 years Daily Returns',height = 450, width = 900)
daily_returns_20yr_plot

In [ ]:
#daily_returns_20yr_mean.sort_values().plot(figsize = (20, 10))
#plt.title('Average Daily Returns')
#plt.xlabel('Percent')
daily_returns_20yr_mean.hvplot.bar(title='All Stocks 20 Years Average Daily Returns',height = 450, width = 750)

In [ ]:
daily_returns_correlation = daily_returns_20yr.corr()
#fig = plt.figure(figsize=(15,10))
#sns.heatmap(daily_returns_correlation, vmin=-1, vmax=1)
daily_returns_correlation.hvplot(kind='heatmap',title='All Stocks 20 Years Daily Returns Correlation', height = 450, width = 750)

In [ ]:
# Box plot to visually show risk
#cumulative_returns.plot.box(figsize=(20,10))

#plt.title("cumulative returns in form of a box presentation of all the funds and SPY")
#plt.ylim(-100, 1150)
#plt.show()


boxplot = cumulative_returns.hvplot.box(title='Box Plot', height = 450, width = 750, legend=False)
boxplot

In [ ]:
#plot dataframe
historical_PE = all_historical_PE_df.hvplot(
                y = ['AAPL', "AMZN",'DAL', 'GE', 'JNJ',"MSFT","MNST","PYPL","SHOP","UPS","SPY"],
                width=950, height=450,rot=90,
                title="P/E, long-term trends, comparison to S&P 500")  
historical_PE

In [ ]:
# Plot cumulative returns
#cumulative_returns.plot(figsize=(20,10))
#plt.title("Daily returns of all the funds, stocks and EFTs")
cumulative_returns.hvplot.line(title='20 years Cumulative Daily Returns of all Stocks', height = 450, width = 750)

In [ ]:
# Calculate the correlation
correlation = cumulative_returns.corr()
# Display de correlation matrix
#sns.heatmap(correlation, vmin=-3, vmax=3, annot=True)
#correlation
#plt.title("Heat map of correlation for all the funds and SPY")
correlation.hvplot(kind='heatmap',title="Heat Map of Correlation for all Stocks and SPY", height = 450, width = 750)

In [ ]:
# Create the main dashboard
dashboard2  = pn.Tabs(
            ("Daily Returns", daily_returns_20yr_plot),
            ("Sharpe Ratios",sharpe_ratios),            
)
dashboard2.servable()

In [ ]:
# Create the main dashboard
dashboard3 = pn.Tabs(
           ("AAPL PE Ratio",AAPL_PE))

dashboard3.servable()

# Comparison of investment of $2000 vs $200,000
### Write here 